# 🔧 Feature Engineering Avancé

Ce notebook se concentre sur la création de features avancées pour l'analyse et la modélisation ML.

## Objectifs
- Calculer les métriques avancées (1RM, volume cumulé, etc.)
- Créer des indicateurs de progression
- Préparer les features pour les modèles ML
- Analyser la distribution et qualité des nouvelles features
- Identifier les features les plus importantes

## 🔧 Imports et configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_regression
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)

print("🔧 Feature Engineering - Environnement initialisé")

## 📁 Chargement et préparation des données

In [ ]:
# Chargement des données
df_raw = pd.read_csv('../examples/sample_data.csv')

# Préparation complète (reprise des notebooks précédents)
df = df_raw.copy()
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Poids_kg'] = df['Poids / Distance'].str.replace(' kg', '').str.replace(',', '.').astype(float)
df['Reps'] = df['Répétitions / Temps'].str.extract(r'(\d+)').astype(float)
df['Volume'] = df['Poids_kg'] * df['Reps']
df['Type_serie'] = df['Série / Série d\'échauffement / Série de récupération']
df['Sautee'] = df['Sautée'].map({'Oui': True, 'Non': False})

# Informations temporelles
df['Jour_Numero'] = df['Date'].dt.dayofweek
df['Semaine'] = df['Date'].dt.isocalendar().week
df['Mois'] = df['Date'].dt.month

# Tri par date et exercice
df = df.sort_values(['Exercice', 'Date']).reset_index(drop=True)

print(f"🔧 Données préparées: {len(df)} sets")
print(f"📊 Exercices uniques: {df['Exercice'].nunique()}")
print(f"📅 Période: {df['Date'].min().strftime('%d/%m/%Y')} → {df['Date'].max().strftime('%d/%m/%Y')}")

## 💪 Calcul du 1RM (One Rep Max)

In [ ]:
# Calcul du 1RM avec différentes formules
print("💪 CALCUL DU 1RM (ONE REP MAX)")
print("=" * 50)

def calculate_1rm_epley(weight, reps):
    """Formule d'Epley: 1RM = poids × (1 + reps/30)"""
    if reps <= 0 or weight <= 0:
        return np.nan
    return weight * (1 + reps / 30)

def calculate_1rm_brzycki(weight, reps):
    """Formule de Brzycki: 1RM = poids × 36 / (37 - reps)"""
    if reps <= 0 or reps >= 37 or weight <= 0:
        return np.nan
    return weight * 36 / (37 - reps)

def calculate_1rm_lombardi(weight, reps):
    """Formule de Lombardi: 1RM = poids × reps^0.10"""
    if reps <= 0 or weight <= 0:
        return np.nan
    return weight * (reps ** 0.10)

# Application des formules
df['1RM_Epley'] = df.apply(lambda row: calculate_1rm_epley(row['Poids_kg'], row['Reps']), axis=1)
df['1RM_Brzycki'] = df.apply(lambda row: calculate_1rm_brzycki(row['Poids_kg'], row['Reps']), axis=1)
df['1RM_Lombardi'] = df.apply(lambda row: calculate_1rm_lombardi(row['Poids_kg'], row['Reps']), axis=1)

# 1RM moyen (moyenne des 3 formules)
df['1RM_Moyen'] = df[['1RM_Epley', '1RM_Brzycki', '1RM_Lombardi']].mean(axis=1)

# Filtrer les séries principales pour le 1RM (exclure échauffement)
df_main_sets = df[df['Type_serie'] == 'Série'].copy()

print("📊 Aperçu des calculs 1RM:")
oneRM_sample = df_main_sets[['Date', 'Exercice', 'Poids_kg', 'Reps', '1RM_Epley', '1RM_Brzycki', '1RM_Lombardi', '1RM_Moyen']].head(10)
print(oneRM_sample)

print(f"\n📈 Statistiques 1RM (séries principales uniquement):")
oneRM_stats = df_main_sets[['1RM_Epley', '1RM_Brzycki', '1RM_Lombardi', '1RM_Moyen']].describe()
print(oneRM_stats)

In [ ]:
# Visualisation des 1RM par exercice
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('💪 Analyse des 1RM par Exercice', fontsize=16, fontweight='bold')

exercises = df_main_sets['Exercice'].unique()

# 1. Comparaison des formules de 1RM
oneRM_comparison = df_main_sets.groupby('Exercice')[['1RM_Epley', '1RM_Brzycki', '1RM_Lombardi']].mean()
oneRM_comparison.plot(kind='bar', ax=axes[0,0], alpha=0.8)
axes[0,0].set_title('Comparaison des Formules 1RM par Exercice')
axes[0,0].set_ylabel('1RM Moyen (kg)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].legend()

# 2. Évolution du 1RM dans le temps (premier exercice)
if len(exercises) > 0:
    main_exercise = exercises[0]
    exercise_data = df_main_sets[df_main_sets['Exercice'] == main_exercise]
    axes[0,1].plot(exercise_data['Date'], exercise_data['1RM_Moyen'], 
                   marker='o', linewidth=2, markersize=6)
    axes[0,1].set_title(f'Évolution 1RM - {main_exercise}')
    axes[0,1].set_ylabel('1RM (kg)')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].grid(True, alpha=0.3)

# 3. Distribution des 1RM
df_main_sets['1RM_Moyen'].hist(bins=15, ax=axes[1,0], alpha=0.7, color='skyblue', edgecolor='black')
axes[1,0].set_title('Distribution des 1RM Moyens')
axes[1,0].set_xlabel('1RM (kg)')
axes[1,0].set_ylabel('Fréquence')
axes[1,0].axvline(df_main_sets['1RM_Moyen'].mean(), color='red', linestyle='--', 
                 label=f'Moyenne: {df_main_sets["1RM_Moyen"].mean():.1f}kg')
axes[1,0].legend()

# 4. Boxplot 1RM par exercice
if len(exercises) > 1:
    df_main_sets.boxplot(column='1RM_Moyen', by='Exercice', ax=axes[1,1])
    axes[1,1].set_title('Distribution 1RM par Exercice')
    axes[1,1].set_xlabel('Exercice')
    axes[1,1].set_ylabel('1RM (kg)')
else:
    axes[1,1].text(0.5, 0.5, 'Pas assez d\'exercices\npour comparaison', 
                   ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Distribution 1RM par Exercice')

plt.tight_layout()
plt.show()

## 📈 Features de progression et tendances

In [ ]:
# Calcul des features de progression
print("📈 FEATURES DE PROGRESSION ET TENDANCES")
print("=" * 50)

# 1. Numérotation des sessions par exercice
df['Session_Number'] = df.groupby('Exercice').cumcount() + 1

# 2. Maximum personnel atteint (rolling max)
df['Max_Poids_Personnel'] = df.groupby('Exercice')['Poids_kg'].cummax()
df['Max_1RM_Personnel'] = df.groupby('Exercice')['1RM_Moyen'].cummax()
df['Max_Volume_Personnel'] = df.groupby('Exercice')['Volume'].cummax()

# 3. Pourcentage du maximum personnel
df['Pct_Max_Poids'] = (df['Poids_kg'] / df['Max_Poids_Personnel'] * 100).round(1)
df['Pct_Max_1RM'] = (df['1RM_Moyen'] / df['Max_1RM_Personnel'] * 100).round(1)

# 4. Progressions (différences par rapport à la session précédente)
df['Progression_Poids'] = df.groupby('Exercice')['Poids_kg'].diff()
df['Progression_Volume'] = df.groupby('Exercice')['Volume'].diff()
df['Progression_1RM'] = df.groupby('Exercice')['1RM_Moyen'].diff()

# 5. Rolling averages (moyennes mobiles)
for window in [3, 5]:
    df[f'Poids_Rolling_{window}'] = df.groupby('Exercice')['Poids_kg'].rolling(window=window, min_periods=1).mean().reset_index(0, drop=True)
    df[f'Volume_Rolling_{window}'] = df.groupby('Exercice')['Volume'].rolling(window=window, min_periods=1).mean().reset_index(0, drop=True)
    df[f'1RM_Rolling_{window}'] = df.groupby('Exercice')['1RM_Moyen'].rolling(window=window, min_periods=1).mean().reset_index(0, drop=True)

# 6. Tendances (régression linéaire sur les N dernières sessions)
from scipy import stats

def calculate_trend(group, column, window=5):
    """Calcule la tendance (pente) sur les dernières sessions"""
    trends = []
    for i in range(len(group)):
        start_idx = max(0, i - window + 1)
        end_idx = i + 1
        
        if end_idx - start_idx >= 2:
            x_vals = list(range(end_idx - start_idx))
            y_vals = group[column].iloc[start_idx:end_idx].values
            
            if len(y_vals) >= 2 and not np.isnan(y_vals).all():
                slope, _, _, _, _ = stats.linregress(x_vals, y_vals)
                trends.append(slope)
            else:
                trends.append(0)
        else:
            trends.append(0)
    
    return trends

# Application du calcul de tendance
df['Tendance_Poids_5'] = df.groupby('Exercice').apply(lambda x: pd.Series(calculate_trend(x, 'Poids_kg', 5), index=x.index)).reset_index(0, drop=True)
df['Tendance_Volume_5'] = df.groupby('Exercice').apply(lambda x: pd.Series(calculate_trend(x, 'Volume', 5), index=x.index)).reset_index(0, drop=True)

print("✅ Features de progression calculées:")
progression_features = ['Session_Number', 'Max_Poids_Personnel', 'Max_1RM_Personnel', 
                       'Pct_Max_Poids', 'Pct_Max_1RM', 'Progression_Poids', 
                       'Progression_Volume', 'Poids_Rolling_3', 'Volume_Rolling_3',
                       'Tendance_Poids_5', 'Tendance_Volume_5']

for feature in progression_features:
    print(f"   • {feature}")

print(f"\n📊 Aperçu des features de progression:")
progression_sample = df[['Date', 'Exercice', 'Poids_kg', 'Volume'] + progression_features[:6]].head(8)
print(progression_sample)

## 🎯 Features de performance et intensité

In [ ]:
# Calcul des features de performance
print("🎯 FEATURES DE PERFORMANCE ET INTENSITÉ")
print("=" * 50)

# 1. Intensité relative (% du 1RM)
df['Intensite_Relative'] = (df['Poids_kg'] / df['1RM_Moyen'] * 100).round(1)

# 2. Volume par kg de poids corporel (simulé à 80kg)
poids_corporel_simule = 80  # kg
df['Volume_Par_Kg_Corps'] = (df['Volume'] / poids_corporel_simule).round(2)

# 3. Densité d'entraînement (volume par minute - simulé)
# Simulation: 3-5 minutes par set
df['Duree_Set_Estimee'] = np.random.uniform(3, 5, len(df))  # minutes
df['Densite_Volume'] = (df['Volume'] / df['Duree_Set_Estimee']).round(2)

# 4. Fatigue index (dégradation dans la session)
def calculate_fatigue_index(group):
    """Calcule l'index de fatigue pour une session"""
    if len(group) < 2:
        return [0] * len(group)
    
    # Fatigue = % de perte par rapport au premier set de la session
    first_weight = group['Poids_kg'].iloc[0]
    fatigue = [(first_weight - weight) / first_weight * 100 if first_weight > 0 else 0 
               for weight in group['Poids_kg']]
    return fatigue

df['Fatigue_Index'] = df.groupby(['Date', 'Exercice']).apply(lambda x: pd.Series(calculate_fatigue_index(x), index=x.index)).reset_index(0, drop=True)

# 5. Efficacité (volume par répétition)
df['Efficacite_Volume'] = (df['Volume'] / df['Reps']).round(2)

# 6. Consistance (écart par rapport à la moyenne personnelle)
df['Moyenne_Poids_Perso'] = df.groupby('Exercice')['Poids_kg'].expanding().mean().reset_index(0, drop=True)
df['Ecart_Moyenne_Perso'] = ((df['Poids_kg'] - df['Moyenne_Poids_Perso']) / df['Moyenne_Poids_Perso'] * 100).round(1)

# 7. Features de récupération (jours depuis dernière session)
df['Jours_Repos'] = df.groupby('Exercice')['Date'].diff().dt.days
df['Jours_Repos'] = df['Jours_Repos'].fillna(0)  # Premier entraînement = 0 jours

# 8. Classification des zones d'intensité
def classify_intensity_zone(intensity):
    """Classifie les zones d'intensité basées sur % 1RM"""
    if pd.isna(intensity):
        return 'Unknown'
    elif intensity < 60:
        return 'Light'  # Échauffement/récupération
    elif intensity < 75:
        return 'Moderate'  # Endurance/volume
    elif intensity < 85:
        return 'Heavy'  # Force
    else:
        return 'Max'  # Maximal

df['Zone_Intensite'] = df['Intensite_Relative'].apply(classify_intensity_zone)

print("✅ Features de performance calculées:")
performance_features = ['Intensite_Relative', 'Volume_Par_Kg_Corps', 'Densite_Volume',
                       'Fatigue_Index', 'Efficacite_Volume', 'Ecart_Moyenne_Perso',
                       'Jours_Repos', 'Zone_Intensite']

for feature in performance_features:
    print(f"   • {feature}")

print(f"\n📊 Distribution des zones d'intensité:")
zone_counts = df['Zone_Intensite'].value_counts()
print(zone_counts)
print(f"\nPourcentages:")
print((zone_counts / len(df) * 100).round(1))

In [ ]:
# Visualisation des features de performance
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('🎯 Analyse des Features de Performance', fontsize=16, fontweight='bold')

# 1. Distribution de l'intensité relative
df['Intensite_Relative'].hist(bins=20, ax=axes[0,0], alpha=0.7, color='lightblue', edgecolor='black')
axes[0,0].set_title('Distribution Intensité Relative (% 1RM)')
axes[0,0].set_xlabel('Intensité (%)')
axes[0,0].set_ylabel('Fréquence')
axes[0,0].axvline(df['Intensite_Relative'].mean(), color='red', linestyle='--', 
                 label=f'Moyenne: {df["Intensite_Relative"].mean():.1f}%')
axes[0,0].legend()

# 2. Zones d'intensité
zone_counts.plot(kind='bar', ax=axes[0,1], color='steelblue', alpha=0.8)
axes[0,1].set_title('Répartition par Zone d\'Intensité')
axes[0,1].set_ylabel('Nombre de Sets')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Fatigue index par exercice
fatigue_by_exercise = df.groupby('Exercice')['Fatigue_Index'].mean()
fatigue_by_exercise.plot(kind='bar', ax=axes[0,2], color='orange', alpha=0.8)
axes[0,2].set_title('Fatigue Index Moyen par Exercice')
axes[0,2].set_ylabel('Fatigue Index (%)')
axes[0,2].tick_params(axis='x', rotation=45)

# 4. Relation intensité vs volume
axes[1,0].scatter(df['Intensite_Relative'], df['Volume'], alpha=0.6, color='green')
axes[1,0].set_title('Relation Intensité vs Volume')
axes[1,0].set_xlabel('Intensité Relative (%)')
axes[1,0].set_ylabel('Volume (kg)')
axes[1,0].grid(True, alpha=0.3)

# 5. Jours de repos vs performance
rest_performance = df.groupby('Jours_Repos')['Intensite_Relative'].mean()
rest_performance.plot(kind='line', marker='o', ax=axes[1,1], color='purple')
axes[1,1].set_title('Jours de Repos vs Performance')
axes[1,1].set_xlabel('Jours de Repos')
axes[1,1].set_ylabel('Intensité Moyenne (%)')
axes[1,1].grid(True, alpha=0.3)

# 6. Efficacité par exercice
efficacite_by_exercise = df.groupby('Exercice')['Efficacite_Volume'].mean()
efficacite_by_exercise.plot(kind='bar', ax=axes[1,2], color='darkred', alpha=0.8)
axes[1,2].set_title('Efficacité Volume par Exercice')
axes[1,2].set_ylabel('Efficacité (kg/rep)')
axes[1,2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🧮 Features temporelles et cycliques

In [ ]:
# Features temporelles et cycliques
print("🧮 FEATURES TEMPORELLES ET CYCLIQUES")
print("=" * 50)

# 1. Features cycliques pour jours de la semaine
df['Jour_Sin'] = np.sin(2 * np.pi * df['Jour_Numero'] / 7)
df['Jour_Cos'] = np.cos(2 * np.pi * df['Jour_Numero'] / 7)

# 2. Features cycliques pour mois
df['Mois_Sin'] = np.sin(2 * np.pi * df['Mois'] / 12)
df['Mois_Cos'] = np.cos(2 * np.pi * df['Mois'] / 12)

# 3. Timestamp normalisé (progression temporelle)
min_date = df['Date'].min()
max_date = df['Date'].max()
df['Timestamp_Norm'] = ((df['Date'] - min_date) / (max_date - min_date)).round(4)

# 4. Features de série dans la session
df['Set_Number_Session'] = df.groupby(['Date', 'Exercice']).cumcount() + 1
df['Total_Sets_Session'] = df.groupby(['Date', 'Exercice'])['Set_Number_Session'].transform('max')
df['Pct_Session_Progress'] = (df['Set_Number_Session'] / df['Total_Sets_Session'] * 100).round(1)

# 5. Cumuls temporels
df['Volume_Cumule_Total'] = df.groupby('Exercice')['Volume'].cumsum()
df['Sessions_Cumule'] = df.groupby('Exercice').cumcount() + 1

# 6. Momentum (accélération des progressions)
df['Momentum_Poids'] = df.groupby('Exercice')['Progression_Poids'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
df['Momentum_Volume'] = df.groupby('Exercice')['Progression_Volume'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)

# 7. Ratios et proportions
# Volume par session (agrégé par jour)
daily_volume = df.groupby(['Date', 'Exercice'])['Volume'].sum().reset_index()
daily_volume.columns = ['Date', 'Exercice', 'Volume_Session']
df = df.merge(daily_volume, on=['Date', 'Exercice'], how='left')

df['Pct_Volume_Session'] = (df['Volume'] / df['Volume_Session'] * 100).round(1)

print("✅ Features temporelles et cycliques calculées:")
temporal_features = ['Jour_Sin', 'Jour_Cos', 'Mois_Sin', 'Mois_Cos', 'Timestamp_Norm',
                    'Set_Number_Session', 'Pct_Session_Progress', 'Volume_Cumule_Total',
                    'Sessions_Cumule', 'Momentum_Poids', 'Momentum_Volume', 'Pct_Volume_Session']

for feature in temporal_features:
    print(f"   • {feature}")

print(f"\n📊 Aperçu des features temporelles:")
temporal_sample = df[['Date', 'Exercice'] + temporal_features[:6]].head(8)
print(temporal_sample)

## 📊 Analyse de corrélation et importance des features

In [ ]:
# Analyse de corrélation des nouvelles features
print("📊 ANALYSE DE CORRÉLATION ET IMPORTANCE DES FEATURES")
print("=" * 60)

# Sélection des features numériques pour l'analyse
numeric_features = ['Poids_kg', 'Reps', 'Volume', '1RM_Moyen', 'Session_Number',
                   'Intensite_Relative', 'Fatigue_Index', 'Efficacite_Volume',
                   'Jours_Repos', 'Progression_Poids', 'Progression_Volume',
                   'Pct_Max_Poids', 'Volume_Cumule_Total', 'Momentum_Poids']

# Filtrer les features qui existent dans le DataFrame
existing_features = [f for f in numeric_features if f in df.columns]
correlation_matrix = df[existing_features].corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('🔍 Matrice de Corrélation des Features Principales', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Identification des corrélations fortes
print("\n🔗 Corrélations significatives (|r| > 0.7):")
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.7:
            feature1 = correlation_matrix.columns[i]
            feature2 = correlation_matrix.columns[j]
            high_corr_pairs.append((feature1, feature2, corr_value))
            print(f"   • {feature1} ↔ {feature2}: {corr_value:.3f}")

if not high_corr_pairs:
    print("   Aucune corrélation > 0.7 détectée")

In [ ]:
# Analyse de l'importance des features avec Information Mutuelle
print("\n🎯 IMPORTANCE DES FEATURES (INFORMATION MUTUELLE)")
print("=" * 60)

# Préparation des données pour l'analyse d'importance
# Target: progression du poids (binaire - progression vs non-progression)
df_analysis = df.dropna(subset=['Progression_Poids']).copy()
df_analysis['Target_Progression'] = (df_analysis['Progression_Poids'] > 0).astype(int)

# Features pour l'analyse d'importance
feature_cols = ['Session_Number', 'Intensite_Relative', 'Jours_Repos', 
               'Fatigue_Index', 'Pct_Max_Poids', 'Volume_Cumule_Total',
               'Momentum_Poids', 'Set_Number_Session', 'Jour_Sin', 'Jour_Cos']

# Filtrer les features existantes et sans NaN
available_features = []
for col in feature_cols:
    if col in df_analysis.columns and not df_analysis[col].isna().all():
        available_features.append(col)

if len(available_features) > 0 and len(df_analysis) > 10:
    X = df_analysis[available_features].fillna(0)
    y = df_analysis['Target_Progression']
    
    # Calcul de l'information mutuelle
    mi_scores = mutual_info_regression(X, y, random_state=42)
    
    # Création du DataFrame des scores
    importance_df = pd.DataFrame({
        'Feature': available_features,
        'Importance_Score': mi_scores
    }).sort_values('Importance_Score', ascending=False)
    
    print("📊 Importance des features pour prédire la progression:")
    print(importance_df)
    
    # Visualisation de l'importance
    plt.figure(figsize=(12, 6))
    plt.bar(importance_df['Feature'], importance_df['Importance_Score'], 
            color='steelblue', alpha=0.8)
    plt.title('🎯 Importance des Features pour Prédire la Progression', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Features')
    plt.ylabel('Score d\'Importance (Information Mutuelle)')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\n🏆 Top 3 des features les plus importantes:")
    for i in range(min(3, len(importance_df))):
        feature = importance_df.iloc[i]['Feature']
        score = importance_df.iloc[i]['Importance_Score']
        print(f"   {i+1}. {feature}: {score:.4f}")
        
else:
    print("⚠️ Pas assez de données pour l'analyse d'importance")

## 💾 Sauvegarde et résumé final

In [ ]:
# Résumé final de toutes les features créées
print("💾 RÉSUMÉ FINAL DES FEATURES CRÉÉES")
print("=" * 60)

# Comptage des features par catégorie
original_features = ['Date', 'Exercice', 'Poids_kg', 'Reps', 'Volume', 'Type_serie', 'Région']
oneRM_features = ['1RM_Epley', '1RM_Brzycki', '1RM_Lombardi', '1RM_Moyen']
progression_features_final = ['Session_Number', 'Max_Poids_Personnel', 'Max_1RM_Personnel', 
                             'Pct_Max_Poids', 'Pct_Max_1RM', 'Progression_Poids', 
                             'Progression_Volume', 'Progression_1RM']
performance_features_final = ['Intensite_Relative', 'Volume_Par_Kg_Corps', 'Densite_Volume',
                             'Fatigue_Index', 'Efficacite_Volume', 'Ecart_Moyenne_Perso',
                             'Jours_Repos', 'Zone_Intensite']
temporal_features_final = ['Jour_Sin', 'Jour_Cos', 'Mois_Sin', 'Mois_Cos', 'Timestamp_Norm',
                          'Set_Number_Session', 'Pct_Session_Progress', 'Volume_Cumule_Total',
                          'Sessions_Cumule', 'Momentum_Poids', 'Momentum_Volume']

# Rolling features
rolling_features = [col for col in df.columns if 'Rolling' in col]
trend_features = [col for col in df.columns if 'Tendance' in col]

print(f"📊 INVENTAIRE DES FEATURES:")
print(f"   • Features originales: {len(original_features)}")
print(f"   • Features 1RM: {len(oneRM_features)}")
print(f"   • Features progression: {len(progression_features_final)}")
print(f"   • Features performance: {len(performance_features_final)}")
print(f"   • Features temporelles: {len(temporal_features_final)}")
print(f"   • Features rolling: {len(rolling_features)}")
print(f"   • Features tendance: {len(trend_features)}")

total_features = len(df.columns)
new_features_count = total_features - len(df_raw.columns)
print(f"\n🎯 TOTAL:")
print(f"   • Features totales: {total_features}")
print(f"   • Nouvelles features créées: {new_features_count}")

# Qualité des données finales
print(f"\n✅ QUALITÉ DES DONNÉES FINALES:")
missing_pct = (df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100)
print(f"   • Complétude globale: {100 - missing_pct:.1f}%")
print(f"   • Lignes de données: {len(df)}")
print(f"   • Exercices couverts: {df['Exercice'].nunique()}")
print(f"   • Période couverte: {(df['Date'].max() - df['Date'].min()).days} jours")

# Features avec le plus de valeurs manquantes
missing_by_feature = df.isnull().sum().sort_values(ascending=False)
if missing_by_feature.iloc[0] > 0:
    print(f"\n⚠️ Features avec valeurs manquantes:")
    for feature, missing_count in missing_by_feature.head(5).items():
        if missing_count > 0:
            pct = missing_count / len(df) * 100
            print(f"   • {feature}: {missing_count} ({pct:.1f}%)")
else:
    print(f"\n✅ Aucune valeur manquante détectée")

print(f"\n🚀 PRÊT POUR:")
print(f"   ✅ Modélisation ML (prédiction de progression)")
print(f"   ✅ Analyse de clustering (profils d'entraînement)")
print(f"   ✅ Détection d'anomalies (performances inhabituelles)")
print(f"   ✅ Recommandations personnalisées")
print(f"   ✅ Dashboard avancé avec métriques enrichies")

In [ ]:
# Sauvegarde optionnelle du dataset enrichi
print("💾 SAUVEGARDE DU DATASET ENRICHI")
print("=" * 50)

# Sélection des features les plus importantes pour la sauvegarde
essential_features = [
    # Features de base
    'Date', 'Exercice', 'Région', 'Poids_kg', 'Reps', 'Volume', 'Type_serie',
    # Features 1RM
    '1RM_Moyen', 'Intensite_Relative',
    # Features progression
    'Session_Number', 'Max_Poids_Personnel', 'Pct_Max_Poids', 'Progression_Poids',
    # Features performance
    'Fatigue_Index', 'Efficacite_Volume', 'Jours_Repos', 'Zone_Intensite',
    # Features temporelles
    'Volume_Cumule_Total', 'Momentum_Poids', 'Set_Number_Session'
]

# Filtrer les features qui existent
available_essential = [f for f in essential_features if f in df.columns]
df_export = df[available_essential].copy()

# Informations sur l'export
print(f"📊 Dataset d'export préparé:")
print(f"   • Features sélectionnées: {len(available_essential)}")
print(f"   • Lignes: {len(df_export)}")
print(f"   • Taille estimée: {df_export.memory_usage(deep=True).sum() / 1024:.1f} KB")

print(f"\n📋 Features incluses dans l'export:")
for i, feature in enumerate(available_essential, 1):
    print(f"   {i:2d}. {feature}")

# Aperçu du dataset final
print(f"\n🔍 Aperçu du dataset enrichi:")
print(df_export.head())

print(f"\n✅ Dataset prêt pour export vers:")
print(f"   • CSV: ../data/features_enriched.csv")
print(f"   • Base de données pour API")
print(f"   • Notebooks de modélisation ML")

---
## 📝 Résumé du Feature Engineering

Ce notebook a créé un ensemble complet de features avancées pour l'analyse de musculation :

### ✅ Features créées
- **💪 1RM & Intensité** : 4 formules de calcul + zones d'intensité
- **📈 Progression** : Tendances, maximums personnels, pourcentages
- **🎯 Performance** : Fatigue, efficacité, densité, récupération
- **⏱️ Temporelles** : Cycliques, cumuls, momentum
- **🔄 Rolling** : Moyennes mobiles sur différentes fenêtres

### 🎯 Insights clés
- **Nombreuses nouvelles features** générées automatiquement
- **Corrélations analysées** pour éviter la redondance
- **Importance évaluée** pour la prédiction de progression
- **Qualité validée** avec gestion des valeurs manquantes

### 📊 Applications possibles
- **Prédiction de performance** future
- **Détection de plateaux** automatisée
- **Recommandations personnalisées** de charge
- **Clustering** de profils d'entraînement
- **Dashboard enrichi** avec métriques avancées

**Dataset prêt pour la Phase 3 (Modèles ML) et les phases suivantes !**